In [1]:
import torch
from torch import nn
import torch.nn.utils.prune as prune
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square conv kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5x5 image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, int(x.nelement() / x.shape[0]))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# 定义一个模型对象
model = LeNet().to(device=device)

In [2]:
# 模型的第一层 六个3*3的卷积核
module = model.conv1
# 这里是函数
print(list(module.named_parameters())) # 打印了权重weight和偏差bias
print(list(module.named_buffers())) # 目前打印为空


[('weight', Parameter containing:
tensor([[[[-0.2208, -0.1358,  0.0358],
          [ 0.1709, -0.1481,  0.0807],
          [ 0.0473, -0.2835, -0.0428]]],


        [[[ 0.3225,  0.3260,  0.1842],
          [-0.1492, -0.3042, -0.1112],
          [-0.2454, -0.2889, -0.3324]]],


        [[[-0.1071,  0.1120,  0.1521],
          [ 0.0496,  0.3321, -0.0295],
          [-0.3143, -0.0115, -0.1824]]],


        [[[-0.0187,  0.1187,  0.2150],
          [-0.0522, -0.1837,  0.1312],
          [-0.1356, -0.0675,  0.2507]]],


        [[[-0.3119, -0.1307,  0.0213],
          [ 0.0547,  0.1024,  0.2038],
          [-0.1779, -0.2047, -0.2900]]],


        [[[ 0.1869, -0.1298,  0.3185],
          [ 0.2846, -0.3239, -0.1929],
          [ 0.2458, -0.0701,  0.0273]]]], device='cuda:0', requires_grad=True)), ('bias', Parameter containing:
tensor([ 0.2520,  0.2080, -0.1745,  0.0363, -0.0401,  0.2278], device='cuda:0',
       requires_grad=True))]
[]


In [4]:
prune.random_unstructured(module, name="weight", amount=0.3) # 裁剪单个module，这里裁剪了个卷积层

print(list(module.named_parameters())) # 参数与原来是相同的，但是不是weight，而是weight_orig

print(list(module.named_buffers())) # 产生一个weight_mask的掩码，产生一个weight的属性

print(module.weight) # 这里是属性

print(module._forward_pre_hooks)

[('bias', Parameter containing:
tensor([ 0.2520,  0.2080, -0.1745,  0.0363, -0.0401,  0.2278], device='cuda:0',
       requires_grad=True)), ('weight_orig', Parameter containing:
tensor([[[[-0.2208, -0.1358,  0.0358],
          [ 0.1709, -0.1481,  0.0807],
          [ 0.0473, -0.2835, -0.0428]]],


        [[[ 0.3225,  0.3260,  0.1842],
          [-0.1492, -0.3042, -0.1112],
          [-0.2454, -0.2889, -0.3324]]],


        [[[-0.1071,  0.1120,  0.1521],
          [ 0.0496,  0.3321, -0.0295],
          [-0.3143, -0.0115, -0.1824]]],


        [[[-0.0187,  0.1187,  0.2150],
          [-0.0522, -0.1837,  0.1312],
          [-0.1356, -0.0675,  0.2507]]],


        [[[-0.3119, -0.1307,  0.0213],
          [ 0.0547,  0.1024,  0.2038],
          [-0.1779, -0.2047, -0.2900]]],


        [[[ 0.1869, -0.1298,  0.3185],
          [ 0.2846, -0.3239, -0.1929],
          [ 0.2458, -0.0701,  0.0273]]]], device='cuda:0', requires_grad=True))]
[('weight_mask', tensor([[[[1., 0., 0.],
          [1., 0

In [6]:
prune.ln_structured(module, name="weight", amount=0.5, n=2, dim=0)

for hook in module._forward_pre_hooks.values():
    if hook._tensor_name == "weight":
       break
print(list(hook))

[<torch.nn.utils.prune.RandomUnstructured object at 0x7f06545835b0>, <torch.nn.utils.prune.RandomUnstructured object at 0x7f0654782eb0>, <torch.nn.utils.prune.LnStructured object at 0x7f0654782e80>, <torch.nn.utils.prune.LnStructured object at 0x7f0544f16580>]


In [15]:
model = LeNet()

parameters_to_prune = (
    (model.conv1, 'weight'),
    (model.conv2, 'weight'),
    (model.fc1, 'weight'),
    (model.fc2, 'weight'),
    (model.fc3, 'weight'),
)

prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.2,
)


print(
    "Sparsity in conv1.weight: {:.2f}%".format(\
        100\. * float(torch.sum(model.conv1.weight == 0))
        / float(model.conv1.weight.nelement())
    )
)
print(
    "Sparsity in conv2.weight: {:.2f}%".format(
        100\. * float(torch.sum(model.conv2.weight == 0))
        / float(model.conv2.weight.nelement())
    )
)
print(
    "Sparsity in fc1.weight: {:.2f}%".format(
        100\. * float(torch.sum(model.fc1.weight == 0))
        / float(model.fc1.weight.nelement())
    )
)
print(
    "Sparsity in fc2.weight: {:.2f}%".format(
        100\. * float(torch.sum(model.fc2.weight == 0))
        / float(model.fc2.weight.nelement())
    )
)
print(
    "Sparsity in fc3.weight: {:.2f}%".format(
        100\. * float(torch.sum(model.fc3.weight == 0))
        / float(model.fc3.weight.nelement())
    )
)
print(
    "Global sparsity: {:.2f}%".format(
        100\. * float(
            torch.sum(model.conv1.weight == 0)
            + torch.sum(model.conv2.weight == 0)
            + torch.sum(model.fc1.weight == 0)
            + torch.sum(model.fc2.weight == 0)
            + torch.sum(model.fc3.weight == 0)
        )
        / float(
            model.conv1.weight.nelement()
            + model.conv2.weight.nelement()
            + model.fc1.weight.nelement()
            + model.fc2.weight.nelement()
            + model.fc3.weight.nelement()
        )
    )
)

SyntaxError: unexpected character after line continuation character (1218652873.py, line 20)